#### Import libraries

In [20]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics


In [21]:
database = pd.read_csv('../scraping/datasets per page/dataframe0.csv')

#### Build total dataset from datasets folder

In [22]:
for x in range(243):
    new_data = pd.read_csv('../scraping/datasets per page/dataframe'+str(x+1)+'.csv')
    database = pd.concat([database, new_data])

In [23]:
df = database
df.shape

(7272, 8)

In [24]:
df = df.reset_index()

#### Drop rows with "None" at sqft_surface

In [25]:
df = df[df['sqft_surface']!='None']

#### Drop duplicates

In [26]:
df_noduplicates = df.drop_duplicates()
df_noduplicates.shape

(7100, 9)

#### Droping disturbing elements in numerical columns

In [27]:
str_bed = []
str_bath = []

for x in df_noduplicates['bedrooms']:
    y = str(x)
    str_bed.append(y)
for x in df_noduplicates['bathrooms']: # convert elements to string
    y = str(x)
    str_bath.append(y)
    
df_noduplicates['bedrooms'] = str_bed
df_noduplicates['bathrooms'] = str_bath # define column elements as string

bed_todrop = df_noduplicates[df_noduplicates['bedrooms'].map(len) > 1]
bath_todrop = df_noduplicates[df_noduplicates['bathrooms'].map(len) > 1] # select the elements to drop

index_todrop = list(bed_todrop.index)
index_todrop2 = list(bath_todrop.index)


for x in index_todrop:
    df_noduplicates = df_noduplicates.drop([x])


for x in index_todrop2:
    try:
        df_noduplicates = df_noduplicates.drop([x])
    except:
        pass
df_noduplicates.head()

,index,address,district,neighbourhood,sqft_surface,bedrooms,bathrooms,new_construction,price
0,0,"Piso en Avinguda del CID, cerca de Plaça de Pe...",Soternes,L'Olivereta,100,3,1,None,104.0
1,1,Piso en Sant Isidre,Sant Isidre,Patraix,105,3,2,None,129.7
2,2,Piso en Els Orriols,Orriols,Rascanya,79,3,1,None,69.0
3,3,"Piso en Carrer de Pedro Cabanes, cerca de Aven...",Sant Antoni,La Saïdia,68,3,1,None,63.5
4,4,"Casa en calle de Escalante, cerca de Calle del...",El Cabanyal-El Canyamelar,Poblats Marítims,42,1,1,None,100.0


#### Correcting address column

In [28]:
df_noduplicates['address'] = df_noduplicates['address'].apply(lambda x: x.split(',')[0])
df_noduplicates['address'] = df_noduplicates['address'].apply(lambda x: x.split(' en ')[1])
df_noduplicates['address'] = df_noduplicates['address'].apply(lambda x: x.strip("'"))
df_noduplicates['address'] = df_noduplicates['address'].apply(lambda x: x.split("-")[0])

In [29]:
to_try = df_noduplicates.drop(columns=['index'])

In [30]:
to_try['bathrooms'].value_counts()

1    3495
2    3264
3     220
4      47
5       3
6       1
Name: bathrooms, dtype: int64

#### Droping unclear addresses

In [31]:
indexNames = to_try[to_try['neighbourhood'] == 'Capital)' ].index
to_try.drop(indexNames , inplace=True)

In [36]:
index = to_try[to_try['district']=='None'].index
to_try.drop(index, inplace = True)
index2 = to_try[to_try['district']=='Pobles del Nord'].index
to_try.drop(index2, inplace = True)
index3 = to_try[to_try['district']=='Pobles del Sud'].index
to_try.drop(index3, inplace = True)
index4 = to_try[to_try['district']=="Pobles de l'Oest"].index
to_try.drop(index4, inplace = True)

#### Droping properties where sqft_living is bellow 10 as they're considered mistakes.

In [38]:
index = to_try[to_try['sqft_surface']<10].index
to_try.drop(index, inplace = True)

#### Multiplying price to get it's real value

In [34]:
to_try['price'] = to_try['price'].apply(lambda x: x*1000)

#### Converting to numerical

In [ ]:
to_try['price'] = pd.to_numeric(to_try['price'])
to_try['sqft_surface'] = pd.to_numeric(to_try['sqft_surface'])
to_try['bedrooms'] = pd.to_numeric(to_try['bedrooms'])
to_try['bathrooms'] = pd.to_numeric(to_try['bathrooms'])

#### Save dataset to csv

In [19]:
df_noduplicates.to_csv('master_data.csv', index=False)
df_noduplicates.head()

,index,address,district,neighbourhood,sqft_surface,bedrooms,bathrooms,new_construction,price
0,0,Avinguda del CID,Soternes,L'Olivereta,100,3,1,None,104.0
1,1,Sant Isidre,Sant Isidre,Patraix,105,3,2,None,129.7
2,2,Els Orriols,Orriols,Rascanya,79,3,1,None,69.0
3,3,Carrer de Pedro Cabanes,Sant Antoni,La Saïdia,68,3,1,None,63.5
4,4,calle de Escalante,El Cabanyal-El Canyamelar,Poblats Marítims,42,1,1,None,100.0
